In [1]:
from config import TrainConfig
import torch
import gc
import pandas as pd
import numpy as np
import os

In [2]:
train_config = TrainConfig()

# Data Loader

In [3]:
from data import LexGlueDataModule

/home/david/anaconda3/envs/r_unfair/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
datamodule = LexGlueDataModule(
    pretrained_model=train_config.pretrained_model,
    max_length=train_config.max_length,
    batch_size=train_config.batch_size,
    num_workers=train_config.num_workers,
    debug_mode_sample=train_config.debug_mode_sample,
)

In [5]:
# Generate a batch
datamodule.setup("fit")

# Create a DataLoader with no parallelization
data_loader = datamodule.train_dataloader()

In [6]:
# Fetch a single batch
for batch_idx, batch in enumerate(data_loader):
    input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["label"]
    print(f"Batch {batch_idx + 1}:")
    print(f"Input IDs: {input_ids.shape}")
    print(f"Attention Mask: {attention_mask.shape}")
    print(f"Labels: {labels.shape}")
    break  # Only fetch the first batch

/home/david/anaconda3/envs/r_unfair/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  self.pid = os.fork()
/home/david/anaconda3/envs/r_unfair/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The mo

Batch 1:
Input IDs: torch.Size([256, 128])
Attention Mask: torch.Size([256, 128])
Labels: torch.Size([256])


In [7]:
labels

tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

# Loading PEFT params

In [8]:
import torch
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType

In [9]:
# Example usage
pretrained_model_name = train_config.pretrained_model
lora_weights_path = "model-checkpoints/lora_adapters.pt"
num_classes = 2

### Inspect LORA Adapters Saved

In [10]:
# Load the saved LoRA adapter weights
lora_weights = torch.load(lora_weights_path, map_location="cpu")

/tmp/ipykernel_720231/3665936600.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_weights = torch.load(lora_weights_path, map_location="cpu")


In [11]:
# Inspect the keys and shapes of the tensors
print("Keys and shapes of tensors in the LoRA adapters:")
for key, tensor in lora_weights.items():
    # print(f"Name: {key}, Shape: {tensor.shape}, Data Type: {tensor.dtype}")
    break


Keys and shapes of tensors in the LoRA adapters:


### Load Base Model

In [12]:
# Load the base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=pretrained_model_name,
    num_labels=num_classes,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Add LORA adapter layers

In [13]:
# Apply LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=True,  # Set to True for inference
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(base_model, peft_config)

In [14]:
# Overwrite the adapter weights
model.load_state_dict(lora_weights, strict=False)

_IncompatibleKeys(missing_keys=['base_model.model.roberta.embeddings.word_embeddings.weight', 'base_model.model.roberta.embeddings.position_embeddings.weight', 'base_model.model.roberta.embeddings.token_type_embeddings.weight', 'base_model.model.roberta.embeddings.LayerNorm.weight', 'base_model.model.roberta.embeddings.LayerNorm.bias', 'base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer.weight', 'base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer.bias', 'base_model.model.roberta.encoder.layer.0.attention.self.key.weight', 'base_model.model.roberta.encoder.layer.0.attention.self.key.bias', 'base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.weight', 'base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.bias', 'base_model.model.roberta.encoder.layer.0.attention.output.dense.weight', 'base_model.model.roberta.encoder.layer.0.attention.output.dense.bias', 'base_model.model.roberta.encoder.layer.0.attentio

### Generate predictions

In [15]:
import torch
from torchmetrics.functional import accuracy

In [16]:
# Ensure model is in evaluation mode
model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [17]:
# Disable gradient computation for inference
with torch.no_grad():
    # Pass the batch through the model
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
    )
    
    # Extract logits
    logits = outputs.logits  # Shape: [batch_size, num_classes]
    
    # Convert logits to predicted class labels
    predictions = torch.argmax(logits, dim=-1)  # Shape: [batch_size]


In [18]:
# Compare predicted labels with true labels
true_labels = labels  # Shape: [batch_size]

num_classes = train_config.num_classes  # Assuming the model config holds the number of labels


# Compute accuracy
accuracy_value = accuracy(predictions, true_labels, task="multiclass", num_classes=num_classes)

In [19]:
# Print results
#print(f"Predicted Labels: {predictions}")
#print(f"True Labels: {true_labels}")
print(f"Accuracy%: {accuracy_value.item() * 100:.2f}")

Accuracy%: 99.61
